In [ ]:
# load the dataset

import pandas as pd

cancer  = pd.read_csv("cancer.csv")


#Loading Data

In [ ]:
cancer .head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,2
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,2
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,2
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,2


In [ ]:
cancer .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      569 non-null    float64
 1   V2      569 non-null    float64
 2   V3      569 non-null    float64
 3   V4      569 non-null    float64
 4   V5      569 non-null    float64
 5   V6      569 non-null    float64
 6   V7      569 non-null    float64
 7   V8      569 non-null    float64
 8   V9      569 non-null    float64
 9   V10     569 non-null    float64
 10  V11     569 non-null    float64
 11  V12     569 non-null    float64
 12  V13     569 non-null    float64
 13  V14     569 non-null    float64
 14  V15     569 non-null    float64
 15  V16     569 non-null    float64
 16  V17     569 non-null    float64
 17  V18     569 non-null    float64
 18  V19     569 non-null    float64
 19  V20     569 non-null    float64
 20  V21     569 non-null    float64
 21  V22     569 non-null    float64
 22  V2

In [ ]:
cancer['Class'].value_counts()

1    357
2    212
Name: Class, dtype: int64

In [ ]:
X = cancer.drop(['Class'],axis=1)

In [ ]:
y = cancer['Class']

#Train And Test Splitting

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(455, 30) (114, 30) (455,) (114,)


In [29]:
# The full pipeline as a step in another pipeline with an estimator as the final step

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

pipe = Pipeline(steps = [("clf",RandomForestClassifier())])
seed = 42
num_folds = 10
scoring = "accuracy"

# create a dictionary with the hyperparameters
search_space = [{"clf":[DecisionTreeClassifier()],
                 "clf__criterion": ["gini","entropy"],
                 "clf__splitter": ["best","random"],
                 "clf__random_state": [seed]},
                 #"fs__score_func":[chi2],
                 #"fs__k":[4,6,8]},
                {"clf":[RandomForestClassifier()],
                 "clf__n_estimators": [200,300],
                 "clf__criterion": ["gini","entropy"],
                 "clf__max_leaf_nodes": [32,64,128],
                 "clf__random_state": [seed]},
                 #"fs__score_func":[chi2],
                 #"fs__k":[4,6,8]},
                {'clf': [BaggingClassifier(DecisionTreeClassifier(random_state=42))],
                 "clf__base_estimator__criterion": ['gini','entropy'],
                 "clf__base_estimator__splitter": ['best','random'],
                 "clf__oob_score": [False],
                 "clf__n_estimators": [200,300],
                 "clf__bootstrap":[False]}]
                 #"fs__score_func":[chi2],
                 #"fs__k":[4,6,8]}]
# create grid search
#kfold = KFold(n_splits=num_folds,random_state=seed)
kfold = KFold(n_splits=num_folds)
grid = GridSearchCV(estimator=pipe, 
                    param_grid=search_space,
                    cv=kfold,
                    scoring=scoring,
                    return_train_score=True,
                    n_jobs=-1,
                    refit="AUC", verbose=10)


# fit grid search
best_model = grid.fit(X_train,y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


In [32]:
print("Best estimator: ",best_model.best_estimator_)
print("==============================================")
print("Best Score: ",best_model.best_score_)

Best estimator:  Pipeline(steps=[('clf',
                 BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                         random_state=42,
                                                                         splitter='random'),
                                   bootstrap=False, n_estimators=300))])
Best Score:  0.971400966183575
